In [ ]:
import apw_path
import plotting
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
%matplotlib inline

fname = 'Laurentia_shape/Laurentia.shp'
laurentia = ShapelyFeature(Reader(fname).geometries(),
            ccrs.PlateCarree(apw_path.lon_shift),
            edgecolor='black', facecolor='#F5DEB3')

In [ ]:
path1, poles, pole_names, pole_colors = apw_path.create_model(0, True)
apw_path.load_or_sample_model(path1)

path2, _, _, _ = apw_path.create_model(1, False)
apw_path.load_or_sample_model(path2)

path3, _, _, _ = apw_path.create_model(1, True)
apw_path.load_or_sample_model(path3)

path4, _, _, _ = apw_path.create_model(2, False)
apw_path.load_or_sample_model(path4)

path5, _, _, _ = apw_path.create_model(2, True)
apw_path.load_or_sample_model(path5)

In [ ]:
fig = plt.figure(figsize=(8,11))

index = 1
def plot_row(path, title):
    global index
    ax1 = fig.add_subplot(4,3,index, projection = ccrs.Orthographic(240.-apw_path.lon_shift, -20))
        
    ax2 = fig.add_subplot(4,3,index+1, projection = ccrs.Orthographic(200-apw_path.lon_shift,30.))
    ax3 = fig.add_subplot(4,3,index+2)
    plotting.plot_synthetic_paths(path, poles, pole_colors, ax1)
    ax1.add_feature(laurentia)
    plotting.plot_synthetic_poles(path, poles, pole_colors, ax2)
    plotting.plot_plate_speeds(path, poles, ax3)
    if index == 1:
        ax1.text(0.5, 1., 'Euler pole distributions',
            horizontalalignment='center',
            verticalalignment='bottom',
            size = 14,
            transform=ax1.transAxes)
        ax2.text(0.5, 1., 'Modeled paleomagnetic poles',
            horizontalalignment='center',
            verticalalignment='bottom',
            size = 14,
            transform=ax2.transAxes)
        ax3.text(0.5, 1., 'Plate speed distributions',
            horizontalalignment='center',
            verticalalignment='bottom',
            size = 14,
            transform=ax3.transAxes)
    ax1.text(0., 0.5, title,
        horizontalalignment='right',
        verticalalignment='center',
        rotation='vertical',
        size=14,
        transform=ax1.transAxes)
    index += 3

plot_row(path1, 'TPW only')
plot_row(path3, 'TPW + one Euler rotation')
plot_row(path4, 'Two Euler rotations')
plot_row(path5, 'TPW + two Euler rotations')
fig.tight_layout()
plt.savefig('many_inversions.pdf')
plt.show()

In [ ]:
fig = plt.figure( figsize = (8,11) )

ax1 = plt.subplot2grid( (6,4), (0,0), colspan=2, rowspan=2, projection=ccrs.Orthographic(220.-apw_path.lon_shift, 30))
ax2 = plt.subplot2grid( (6,4), (0,2), colspan=2, rowspan=2, projection=ccrs.Orthographic(200-apw_path.lon_shift,30.))        
ax3 = plt.subplot2grid( (6,4), (2,0), colspan=2, rowspan=2 )
ax4 = plt.subplot2grid( (6,4), (2,2), colspan=2, rowspan=2 )
ax5 = plt.subplot2grid( (6,4), (4,0), colspan=4, rowspan=1 )
ax6 = plt.subplot2grid( (6,4), (5,0), colspan=4, rowspan=1)

plotting.plot_synthetic_paths(path2, poles, pole_colors, ax1)
ax1.add_feature(laurentia)
plotting.plot_synthetic_poles(path2, poles, pole_colors, ax2)
plotting.plot_plate_speeds(path2, poles, ax3, '')
plotting.make_legend(pole_names, pole_colors, ax4, 'Paleomagnetic poles')
plotting.plot_age_samples(path2, poles, pole_colors, ax5, ax6)
fig.tight_layout()
plt.savefig('one_euler_pole.pdf')
plt.show()


In [ ]:
import seaborn as sns

def make_jointplot(tpw_rate, euler_rate, color, title):
    jp = sns.jointplot(tpw_rate, euler_rate, kind="hex",\
                       xlim=(0,4), ylim=(0,4),
                       stat_func=None, color=color, gridsize=(20,20))
    jp.set_axis_labels("TPW rate ($^\circ$/Myr)", "Plate rotation rate ($^\circ$/Myr)")
    jp.ax_joint.set_axis_bgcolor("white")
    plt.savefig(title+'.pdf')
    

make_jointplot(path3.tpw_rates(), path3.euler_rates()[0], 'darkred', 'one_euler_jointplot')
make_jointplot(path5.tpw_rates(), path5.euler_rates()[0], 'darkred', 'two_euler_jointplot_1')
make_jointplot(path5.tpw_rates(), path5.euler_rates()[1], 'darkgreen', 'two_euler_jointplot_2')
